## 简述
这是图神经网络的最基础的模型GCN，本代码是对由Thomas Kipf和Max Welling在2016年提出的。他们的文章标题为“Semi-Supervised Classification with Graph Convolutional Networks”开源的代码进行一个详解
## 数据集
本代码使用的是cora数据集，
Cora数据集是自然语言处理和图神经网络研究中经常使用的一个数据集，主要用于文献分类任务。以下是对Cora数据集的简介：
内容：Cora数据集包含2,708个科学出版物，这些出版物属于7个类别。
特点：
节点：每个节点代表一篇科学出版物。
边：如果一篇文章引用了另一篇文章，它们之间就有一条边。
特征：每篇出版物都有一个词向量，该向量基于出版物中的词的出现或不出现。原始的Cora数据集使用了一个1433维的二进制词向量来表示每个文档。
标签：数据集中的出版物分为7个类别，包括：“Case_Based”、“Genetic_Algorithms”、“Neural_Networks”、“Probabilistic_Methods”、“Reinforcement_Learning”、“Rule_Learning”和“Theory”。
## 关键点
1.理解图结构的形式
2.如何使用邻接矩阵实现其图结构形式
3.GCN卷积是如何实现节点特征更新的


## 数据预处理

In [4]:
import numpy as np
import scipy.sparse as sp
import torch
##以数组形式提取cora。content
idx_features_labels = np.genfromtxt('C:\\Users\\Wei Zhou\\Desktop\\test\\图神经网络几个算法\\gcn\\data\\cora\\cora.content',dtype=np.dtype(str))
print(idx_features_labels)

[['31336' '0' '0' ... '0' '0' 'Neural_Networks']
 ['1061127' '0' '0' ... '0' '0' 'Rule_Learning']
 ['1106406' '0' '0' ... '0' '0' 'Reinforcement_Learning']
 ...
 ['1128978' '0' '0' ... '0' '0' 'Genetic_Algorithms']
 ['117328' '0' '0' ... '0' '0' 'Case_Based']
 ['24043' '0' '0' ... '0' '0' 'Neural_Networks']]


In [5]:
##提取节点特征
features = sp.csr_matrix(idx_features_labels[:, 1:-1], dtype=np.float32)  #sp.csr_matrix建立稀疏矩阵
print(features.A)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [6]:
## 引入one hot 编码
def encode_onehot(labels):
    classes = set(labels)
    classes_dict = {c: np.identity(len(classes))[i, :] for i, c in
                    enumerate(classes)}
    labels_onehot = np.array(list(map(classes_dict.get, labels)),
                             dtype=np.int32)
    return labels_onehot

In [7]:
## 对标签进行one hot 编码
labels = encode_onehot(idx_features_labels[:, -1]) 
print(labels)

[[0 1 0 ... 0 0 0]
 [1 0 0 ... 0 0 0]
 [0 0 0 ... 0 1 0]
 ...
 [0 0 0 ... 1 0 0]
 [0 0 1 ... 0 0 0]
 [0 1 0 ... 0 0 0]]


In [8]:
## 取节点的索引
idx = np.array(idx_features_labels[:, 0], dtype=np.int32)
print(idx)

[  31336 1061127 1106406 ... 1128978  117328   24043]


In [9]:
#构建节点索引
idx_map = {j: i for i, j in enumerate(idx)} 
print(idx_map)

{31336: 0, 1061127: 1, 1106406: 2, 13195: 3, 37879: 4, 1126012: 5, 1107140: 6, 1102850: 7, 31349: 8, 1106418: 9, 1123188: 10, 1128990: 11, 109323: 12, 217139: 13, 31353: 14, 32083: 15, 1126029: 16, 1118017: 17, 49482: 18, 753265: 19, 249858: 20, 1113739: 21, 48766: 22, 646195: 23, 1126050: 24, 59626: 25, 340299: 26, 354004: 27, 242637: 28, 1106492: 29, 74975: 30, 1152272: 31, 100701: 32, 66982: 33, 13960: 34, 13966: 35, 66990: 36, 182093: 37, 182094: 38, 13972: 39, 13982: 40, 16819: 41, 273152: 42, 237521: 43, 1153703: 44, 32872: 45, 284025: 46, 218666: 47, 16843: 48, 1153724: 49, 1153728: 50, 158098: 51, 8699: 52, 1134865: 53, 28456: 54, 248425: 55, 1112319: 56, 28471: 57, 175548: 58, 696345: 59, 28485: 60, 1139195: 61, 35778: 62, 28491: 63, 310530: 64, 1153784: 65, 1481: 66, 1153786: 67, 13212: 68, 1111614: 69, 5055: 70, 4329: 71, 330148: 72, 1105062: 73, 4330: 74, 5062: 75, 4335: 76, 158812: 77, 40124: 78, 1103610: 79, 688361: 80, 302545: 81, 20534: 82, 1031453: 83, 5086: 84, 193742

In [10]:
##读取边的数据
edges_unordered = np.genfromtxt('C:\\Users\\Wei Zhou\\Desktop\\test\\图神经网络几个算法\\gcn\\data\\cora\\cora.cites',
                                    dtype=np.int32)
print(edges_unordered)

[[     35    1033]
 [     35  103482]
 [     35  103515]
 ...
 [ 853118 1140289]
 [ 853155  853118]
 [ 954315 1155073]]


In [11]:
##构成边的点的原有编码替换成新的点的编码
edges = np.array(list(map(idx_map.get, edges_unordered.flatten())),dtype=np.int32).reshape(edges_unordered.shape)
print(edges)

[[ 163  402]
 [ 163  659]
 [ 163 1696]
 ...
 [1887 2258]
 [1902 1887]
 [ 837 1686]]


In [12]:
##构建邻接矩阵且为稀疏矩阵
adj = sp.coo_matrix((np.ones(edges.shape[0]), (edges[:, 0], edges[:, 1])),
                        shape=(labels.shape[0], labels.shape[0]),
                        dtype=np.float32)
print(adj.A)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [13]:
##计算转置矩阵将有向图转化为无向图
adj = adj + adj.T.multiply(adj.T > adj) - adj.multiply(adj.T > adj)

In [14]:
##归一化函数
def normalize(mx):
    rowsum = np.array(mx.sum(1)) #矩阵行求和
    r_inv = np.power(rowsum, -1).flatten() ##求和的倒数
    r_inv[np.isinf(r_inv)] = 0.#由于原本为0的数求倒数后可能会产生极大值，这里设置极大值为0
    r_mat_inv = sp.diags(r_inv) ##构造对角线矩阵
    mx = r_mat_inv.dot(mx)
    return mx

In [15]:
## 对节点进行归一化
features = normalize(features)
print(features.A)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [16]:
##先将稀疏矩阵转化为密集矩阵再进行归一化操作
adj = normalize(adj + sp.eye(adj.shape[0]))
print(adj.A)

[[0.16666667 0.         0.         ... 0.         0.         0.        ]
 [0.         0.5        0.         ... 0.         0.         0.        ]
 [0.         0.         0.2        ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 0.2        0.         0.        ]
 [0.         0.         0.         ... 0.         0.2        0.        ]
 [0.         0.         0.         ... 0.         0.         0.25      ]]


In [17]:
##SciPy稀疏矩阵转换为PyTorch稀疏张量
def sparse_mx_to_torch_sparse_tensor(sparse_mx):
    sparse_mx = sparse_mx.tocoo().astype(np.float32)
    indices = torch.from_numpy(
        np.vstack((sparse_mx.row, sparse_mx.col)).astype(np.int64))
    values = torch.from_numpy(sparse_mx.data)
    shape = torch.Size(sparse_mx.shape)
    return torch.sparse.FloatTensor(indices, values, shape)

In [18]:
##将scipy的稀疏矩阵转化为密集矩阵，再转化其为tensor形式
features = torch.FloatTensor(np.array(features.todense()))
print(features)

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])


In [19]:
##将标签转化为longtensor形式
labels = torch.LongTensor(np.where(labels)[1])
print(labels)

tensor([1, 0, 5,  ..., 4, 2, 1])


In [20]:
##将邻接矩阵转化为tensor形式
adj = sparse_mx_to_torch_sparse_tensor(adj)
print(adj)

tensor(indices=tensor([[   0,    0,    0,  ..., 2707, 2707, 2707],
                       [   0,    8,   14,  ..., 1389, 2344, 2707]]),
       values=tensor([0.1667, 0.1667, 0.1667,  ..., 0.2500, 0.2500, 0.2500]),
       size=(2708, 2708), nnz=13264, layout=torch.sparse_coo)


In [21]:
##划分数据集
idx_train = range(140)
idx_val = range(200, 500)
idx_test = range(500, 1500)

idx_train = torch.LongTensor(idx_train)
idx_val = torch.LongTensor(idx_val)
idx_test = torch.LongTensor(idx_test)

## 训练前设置

In [22]:
import argparse 
parser = argparse.ArgumentParser()

##--no-cuda: 如果指定了这个选项，将禁用 CUDA 训练。
parser.add_argument('--no-cuda', action='store_true', default=False,
                    help='Disables CUDA training.')
##--fastmode: 如果指定了这个选项，在训练过程中会进行验证。
parser.add_argument('--fastmode', action='store_true', default=False,
                    help='Validate during training pass.')
##--seed: 随机种子，默认值为 42。
parser.add_argument('--seed', type=int, default=42, help='Random seed.')

##--epochs: 要训练的轮数，默认值为 200。
parser.add_argument('--epochs', type=int, default=200,
                    help='Number of epochs to train.')

##--lr: 初始学习率，默认值为 0.01。
parser.add_argument('--lr', type=float, default=0.01,
                    help='Initial learning rate.')

##--weight_decay: 权重衰减（用于 L2 正则化），默认值为 5e-4。
parser.add_argument('--weight_decay', type=float, default=5e-4,
                    help='Weight decay (L2 loss on parameters).')

##--hidden: 隐藏层单元数，默认值为 16。
parser.add_argument('--hidden', type=int, default=16,
                    help='Number of hidden units.')

##--dropout: dropout 率（1 - 保持概率），默认值为 0.5。
parser.add_argument('--dropout', type=float, default=0.5,
                    help='Dropout rate (1 - keep probability).')

##设置随机种子和检查 CUDA 可用性的
args, _ = parser.parse_known_args()
args.cuda = not args.no_cuda and torch.cuda.is_available()

np.random.seed(args.seed)
torch.manual_seed(args.seed)
if args.cuda:
    torch.cuda.manual_seed(args.seed)

## 模型设置

In [23]:
##定义GNN模型
import torch.nn as nn
import torch.nn.functional as F

class GCN(nn.Module):
    def __init__(self, nfeat, nhid, nclass, dropout):
        super(GCN, self).__init__()

        self.gc1 = GraphConvolution(nfeat, nhid)#构建第一层GCN
        self.gc2 = GraphConvolution(nhid, nclass)#构建第二层GCN
        self.dropout = dropout
        
##正向传播
    def forward(self, x, adj):
        x = F.relu(self.gc1(x, adj))#将节点特征矩阵和邻接矩阵通过第一层GCN然后在对进行非线性化
        x = F.dropout(x, self.dropout, training=self.training)##较少过拟合使用drop out
        x = self.gc2(x, adj)##将第一层输出的节点特诊和邻接矩阵输入第二层图卷积
        return F.log_softmax(x, dim=1)##应用 log softmax 函数以获得每个类别的预测概率。

In [24]:
##定义图神经卷积层
import math
import torch
from torch.nn.parameter import Parameter
from torch.nn.modules.module import Module

class GraphConvolution(Module):
    def __init__(self, in_features, out_features, bias=True):
        super(GraphConvolution, self).__init__()
        ##输入特征维度
        self.in_features = in_features
        ##输出特征维度
        self.out_features = out_features
        
        ##初始化权重矩阵。其形状是 (in_features, out_features)，这表示它将一个 in_features 维的向量转换为 out_features 维的向量。
        self.weight = Parameter(torch.FloatTensor(in_features, out_features))
        
        ##初始化偏置项。创建一个长度为out_fetures的向量
        if bias:
            self.bias = Parameter(torch.FloatTensor(out_features))
        else:
            self.register_parameter('bias', None)
        self.reset_parameters()

    def reset_parameters(self):
        ##用于权重和偏置初始化的标准偏差
        stdv = 1. / math.sqrt(self.weight.size(1))
        self.weight.data.uniform_(-stdv, stdv)
        if self.bias is not None:
            self.bias.data.uniform_(-stdv, stdv)

    def forward(self, input, adj):
        ##进行矩阵乘法：输入的节点特征和权重矩阵
        support = torch.mm(input, self.weight)
        ##执行矩阵乘法：稀疏矩阵（adj）和一般矩阵（support）的乘法
        output = torch.spmm(adj, support)
        if self.bias is not None:
            return output + self.bias
        else:
            return output

    def __repr__(self):
        ##输出这个类的有用信息，如输入/输出特征的维度
        return self.__class__.__name__ + ' (' \
               + str(self.in_features) + ' -> ' \
               + str(self.out_features) + ')'

In [25]:
#导入模型
model = GCN(nfeat=features.shape[1],## 特征数量
            nhid=args.hidden,## 隐藏层节点数量
            nclass=labels.max().item() + 1,##类别数量，由标签最大值确定
            dropout=args.dropout)##dropout，防止过拟合

In [26]:
#定义优化器
import torch.optim as optim
optimizer = optim.Adam(model.parameters(),lr=args.lr, weight_decay=args.weight_decay)
#model.parameters(): 这是一个生成器，用于遍历模型的所有可训练参数。这些参数将会被优化器更新
#lr=args.lr: 这里设置了学习率（Learning Rate）。该值从之前定义的 args 对象中获取。学习率是一个控制模型权重更新步长的超参数。
#weight_decay=args.weight_decay: 权重衰减（通常用于 L2 正则化）也是从 args 对象中获取的。权重衰减可以用于防止模型过拟合。

## 模型训练

In [27]:
import time
def train(epoch):
##记录开始训练的时间
    t = time.time()
##调整模型为训练模式
    model.train()
##梯度清零
    optimizer.zero_grad()
##前向传播，将节点特征和邻接矩阵相乘
    output = model(features, adj)
##损失函数计算
    loss_train = F.nll_loss(output[idx_train], labels[idx_train])
##计算准确度
    acc_train = accuracy(output[idx_train], labels[idx_train])
##反向传播计算梯度
    loss_train.backward()
##更新模型参数
    optimizer.step()

##检查是否是快速模式
    if not args.fastmode:
        # Evaluate validation set performance separately,
        # deactivates dropout during validation run.
        model.eval()
        output = model(features, adj)

    loss_val = F.nll_loss(output[idx_val], labels[idx_val])
    acc_val = accuracy(output[idx_val], labels[idx_val])
    print('Epoch: {:04d}'.format(epoch+1),
          'loss_train: {:.4f}'.format(loss_train.item()),
          'acc_train: {:.4f}'.format(acc_train.item()),
          'loss_val: {:.4f}'.format(loss_val.item()),
          'acc_val: {:.4f}'.format(acc_val.item()),
          'time: {:.4f}s'.format(time.time() - t))

In [28]:
##计算准确度的函数
def accuracy(output, labels):
    preds = output.max(1)[1].type_as(labels)##提取输出的的每一行的最大值
    correct = preds.eq(labels).double()##对比预测和标签使用布尔张量表示
    correct = correct.sum()#对上一步生成的浮点数张量求和，以得到预测正确的样本数。
    return correct / len(labels)#正确预测的样本数与总样本数的比值

## 测试模型

In [29]:
def test():
    model.eval()
##将模型调整为评估模式-关闭反向传播
    output = model(features, adj)
##求损失值
    loss_test = F.nll_loss(output[idx_test], labels[idx_test])
##求准确度
    acc_test = accuracy(output[idx_test], labels[idx_test])
    print("Test set results:",
          "loss= {:.4f}".format(loss_test.item()),
          "accuracy= {:.4f}".format(acc_test.item()))

In [30]:
t_total = time.time()
for epoch in range(args.epochs):
    train(epoch)
print("Optimization Finished!")
print("Total time elapsed: {:.4f}s".format(time.time() - t_total))

# Testing
test()

Epoch: 0001 loss_train: 1.9240 acc_train: 0.2286 loss_val: 1.8940 acc_val: 0.3500 time: 0.0615s
Epoch: 0002 loss_train: 1.9163 acc_train: 0.2929 loss_val: 1.8829 acc_val: 0.3500 time: 0.0080s
Epoch: 0003 loss_train: 1.8986 acc_train: 0.2929 loss_val: 1.8725 acc_val: 0.3500 time: 0.0103s
Epoch: 0004 loss_train: 1.8850 acc_train: 0.2929 loss_val: 1.8625 acc_val: 0.3500 time: 0.0000s
Epoch: 0005 loss_train: 1.8798 acc_train: 0.2929 loss_val: 1.8528 acc_val: 0.3500 time: 0.0099s
Epoch: 0006 loss_train: 1.8668 acc_train: 0.2929 loss_val: 1.8432 acc_val: 0.3500 time: 0.0035s
Epoch: 0007 loss_train: 1.8630 acc_train: 0.2929 loss_val: 1.8340 acc_val: 0.3500 time: 0.0070s
Epoch: 0008 loss_train: 1.8500 acc_train: 0.2929 loss_val: 1.8252 acc_val: 0.3500 time: 0.0000s
Epoch: 0009 loss_train: 1.8367 acc_train: 0.2929 loss_val: 1.8167 acc_val: 0.3500 time: 0.0092s
Epoch: 0010 loss_train: 1.8319 acc_train: 0.2929 loss_val: 1.8085 acc_val: 0.3500 time: 0.0000s
Epoch: 0011 loss_train: 1.8283 acc_train